In [1]:
from __future__ import division
from __future__ import print_function

import sys, os,gc
import tensorflow as tf
import numpy as np

sys.path.append('../')

from models.neural_network import NeuralNetwork
from models.cnn import CNN
from models.util import *

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

#Seed used for all calculations of training and test point indices 
SEED = 14

Using TensorFlow backend.


In [2]:
#Define params of model
input_shape = (28,28,1)
num_classes = 10
eps=0.3


In [3]:
#Load model from disk
model_name = 'MNIST'
model_save_path = '../trained_models/' + model_name + '-model.json'
weights_save_path = '../trained_models/' + model_name + 'weights'
model = CNN(model_name=model_name, dataset='mnist', seed=SEED)
print ('Total Model Params: %d' % model.num_params)
model.load_model(model_save_path, weights_save_path) 
#epochs = 50
#model.train(epochs=epochs)
#model.save_model(model_save_path, weights_save_path)   

Total Model Params: 3330314
Loaded model from disk


In [4]:
#Model Accuracy
print ('Model Accuracy: %.5f' % (model.model.evaluate(model.test_data, model.test_labels)[1]))

6055/6055 [==============================] - 2s 306us/step
Model Accuracy: 0.99306


In [5]:
#Get training samples
num_train_samples = 1000
data_indices = model.gen_rand_indices(low=0, high=model.train_data.shape[0], seed=SEED, num_samples=num_train_samples)
train_data = model.train_data[data_indices]
train_data_labels = model.train_labels[data_indices]
train_data_labels_int = np.argmax(train_data_labels, axis=1)


In [6]:
num_test_samples_per_class = 10
num_test_samples = num_classes*num_test_samples_per_class

#Generate test points
test_indices = model.gen_rand_indices_all_classes(y=model.test_labels, seed=SEED, num_samples=num_test_samples_per_class)

#Get Regular, Noisy, FGSM, BIM, and CW test points
reg_data = model.test_data[test_indices]

In [7]:
#Calculate Hessian Vector Product Matrix to be used for Influence
hvp_matrix = np.zeros((reg_data.shape[0], model.num_params))
for idx, img_lab in enumerate(zip(reg_data, model.test_labels[test_indices])):
    inverse_hvp = model.get_inverse_hvp(img_lab[0], img_lab[1])
    hvp_matrix[idx,:] = inverse_hvp


In [8]:
reg_preds = model.model.predict(reg_data.reshape(-1,*input_shape))
reg_labels = preds_to_labels(reg_preds)


In [9]:
#Get Gradients required for Similarity
grads_train = model.get_gradients_wrt_params(train_data, train_data_labels)
grads_reg = model.get_gradients_wrt_params(reg_data, reg_labels)


In [10]:
#Calculate Influence and Similarity Matrices
inf_matrix = np.dot(hvp_matrix, grads_train.T)
sim_matrix = np.dot(grads_reg, grads_train.T)

In [11]:
#Calculate percentage of points for which H^{-1} acts as a scaling
avg_cos = 0.0
scaling_count = 0.0
for i in range(num_test_samples ):
    hvp_norm=np.linalg.norm(hvp_matrix[i])
    reg_norm = np.linalg.norm(grads_reg[i])
    if hvp_norm > 0 and reg_norm > 0:
        a = hvp_matrix[i]/hvp_norm
        b = grads_reg[i]/reg_norm
        cos=np.dot(a,b)
        avg_cos+=cos
        #consder > 0.99 as parallel due to approx errors
        if cos >= 0.99:
             scaling_count+=1.0
    else:
        print ('Divide by zero error, skipping point ...')

print ('Avg Cos Sim: %.5f, Ratio of Points Parallel: %.5f' % ((avg_cos/num_test_samples), (scaling_count/num_test_samples)))


Avg Cos Sim: 0.99984, Ratio of Points Parallel: 1.00000


In [12]:
#Get Avg Scale Constant and get ratio of test points for which (max sim == max inf)
scaling_const = 0.0
arg_max = 0.0
for i in range(num_test_samples):
    if (np.argmax(sim_matrix[i]) == np.argmax(inf_matrix[i])):
        arg_max+=1.0
    for j in range(num_train_samples):
        const = sim_matrix[i,j]/inf_matrix[i,j]
        scaling_const+=const

print ('Avg Scaling Const: %.5f, Ratio of Same Arg Max: %.5f' % ((scaling_const/(num_test_samples*num_train_samples)), (arg_max/num_test_samples)))   

Avg Scaling Const: 99.49869, Ratio of Same Arg Max: 0.97000
